In [ ]:
import cv2 as cv2
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import optimizers, initializers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten,Dropout, MaxPooling2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping, History

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import import_ipynb
import dataSetUtility as dsu
import plottingUtility as pltu

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from keras.utils import np_utils
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
import imageio
from PIL import Image
import tifffile

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import seaborn as sn
import numpy as np
from scipy import ndimage

import random
import PIL
import os
import cv2 as cv2
import itertools
import glob

In [ ]:
#path to the directories
pathTrain=r'C:\Users\39320\Desktop\tesi\Computer-Vision-2020-main\ImageSet\train'
pathTest=r'C:\Users\39320\Desktop\tesi\Computer-Vision-2020-main\ImageSet\test'

labels = [os.path.basename(i) for i in glob.glob(pathTrain + '/*', recursive=True)]
numberOfClasses = len(labels)
print("Class number: ", numberOfClasses)
print("Class names: ", labels)

In [ ]:
def get_random_crop(img, width, height, dim=64):
    assert img.shape[0] >= height
    assert img.shape[1] >= width
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    resImg = cv2.resize(img, (dim,dim), interpolation=cv2.INTER_CUBIC)
    return resImg

In [ ]:
import math
import copy

def warpAffine(src, M, dsize, from_bounding_box_only=False):
    """
    Applies cv2 warpAffine, marking transparency if bounding box only
    The last of the 4 channels is merely a marker. It does not specify opacity in the usual way.
    """
    return cv2.warpAffine(src, M, dsize)

def rotate_image(image, dim=64):
    """Rotate the image counterclockwise.
    Rotate the image such that the rotated image is enclosed inside the
    tightest rectangle. The area not occupied by the pixels of the original
    image is colored black.
    Parameters
    ----------
    image : numpy.ndarray
        numpy image
    angle : float
        angle by which the image is to be rotated. Positive angle is
        counterclockwise.
    Returns
    -------
    numpy.ndarray
        Rotated Image
    """
    angle = random.randint(0, 360)
    # get dims, find center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = warpAffine(image, M, (nW, nH), False)

    resImg = cv2.resize(image, (dim,dim), interpolation=cv2.INTER_CUBIC)

    return resImg

In [ ]:
def dataAugmentationWithCropRot(path, labels, dimension=64, d=1):
    x = [] #New ImageSet
    y = [] #New Label
    for index,name in enumerate(labels):
        s = path+'/'+name+'/{}'
        s2 = path+'/'+name+'/'
        temp = [s.format(i) for i in os.listdir(path+'/'+name+'/')]
        q=0
        for image in temp:
            q = q+1
            tmp = cv2.resize(cv2.imread(image,cv2.IMREAD_GRAYSCALE),(dimension,dimension),interpolation=cv2.INTER_CUBIC)
            x.append(tmp)
            imageio.imwrite(s2 + 'outfile1' + str(q) + str(index) + '.jpg', tmp)
            q = q+1
            x.append(cv2.flip(tmp,1))
            imageio.imwrite(s2+ 'outfile2' + str(q)  + str(index)+  '.jpg', cv2.flip(tmp,1))
            q = q+1
            x.append(get_random_crop(tmp, dimension-24, dimension-24, dimension))
            imageio.imwrite(s2+ 'outfile3' + str(q)+   str(index) + '.jpg', get_random_crop(tmp, dimension-24, dimension-24, dimension))
            q = q+1
            x.append(rotate_image(tmp, dimension))
            imageio.imwrite(s2+ 'outfile4' + str(q) + str(index)+ '.jpg', rotate_image(tmp, dimension))
            q = q+1
            y.append(index)
            y.append(index)
            y.append(index)
            y.append(index)
        
    x = np.asarray([np.reshape(im, (dimension, dimension, d)) for im in x])
    x=x/255
    y = np.asarray(y)
    return x,y

In [ ]:
#xTrain, yTrain = dataAugmentationWithCropRot(pathTrain, labels, dimension=224)

In [ ]:
#xTrain.shape

In [ ]:
xTrain, yTrain = dataAugmentationWithCropRot(pathTrain, labels, dimension=224, d=1)
xTest, yTest = dsu.loadImages(pathTest, labels, dimension=224)

In [ ]:
yTrainDummy = to_categorical(yTrain, 12)
yTestDummy = to_categorical(yTest, 12)

In [ ]:
model = ResNet50(weights='imagenet', include_top=False)
model.summary()

In [ ]:
def extractFeatures(model, path, labels):
    
    pathImages = []
    for label in labels:
        s = path+'/'+label+'/{}'
        temp = [s.format(i) for i in os.listdir(path+'/'+label+'/')]
        pathImages = pathImages + temp
    
    res = []
    count = 0
    for p in pathImages:
        count += 1
        img = load_img(p, target_size=(224,224))
        print(p)
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = imagenet_utils.preprocess_input(img)
        res.append(img)
        
    images = np.vstack(res)
    features = model.predict(images, batch_size=64)
    featuresFlatten = features.reshape((features.shape[0], 7 * 7 * 2048))
    
    return images, features, featuresFlatten

In [ ]:
trainImages, featureMatrix, featureArray = extractFeatures(model, pathTrain, labels)

In [ ]:
testImages, featureMatrixTest, featureArrayTest = extractFeatures(model, pathTest, labels)

In [ ]:
classif = [SVC(kernel="linear") for _ in range(numberOfClasses)]

currentLabel = 0
for clf in classif:
    print(currentLabel)
    v = np.array([1 if label==currentLabel else 0 for label in yTrain])
    clf = clf.fit(featureArray, v)
    currentLabel += 1

In [ ]:
prediction = []
for image in featureArrayTest:
    pred = np.array([np.dot(clf.coef_,image) + clf.intercept_ for clf in classif])
    prediction.append(np.argmax(pred))
    
prediction = np.asarray(prediction)

In [ ]:
cont=0
for i in range(len(prediction)):
    if prediction[i]==yTest[i]:
        cont += 1
    else:
        continue
        
accuracy = cont/len(prediction)
print("Accuracy value: ",accuracy)

In [ ]:
tp = []
fp = []
fn = []
tn = []
for i in range(numberOfClasses):
    tp_temp = 0
    fp_temp = 0
    fn_temp = 0
    tn_temp = 0
    
    for j in range(len(prediction)):
        if(prediction[j]==i and yTest[j]==i):
            tp_temp += 1
        if(prediction[j]==i and yTest[j]!=i):
            fp_temp += 1
        if(prediction[j]!=i and yTest[j]==i):
            fn_temp += 1
        if(prediction[j]!=i and yTest[j]!=i):
            tn_temp += 1
    tp.append(tp_temp)
    fp.append(fp_temp)
    fn.append(fn_temp)
    tn.append(tn_temp)

In [ ]:
data = {'labels':labels , 'True positive':tp,'True negative':tn,'False positive':fp,'False negative':fn}
evaluation = pd.DataFrame(data, columns = ['labels','True positive','True negative','False positive','False negative'])
print("Evaluation: ")
evaluation

In [ ]:
cm = np.zeros((12,12))
for i in range(numberOfClasses):
    cm[i,i] = evaluation.loc[i]['True positive']
    
for i in range(numberOfClasses):
    for j in range(numberOfClasses):
        temp = 0
        for k in range(len(yTest)):
            if(yTest[k]==i and prediction[k]==j):
                temp += 1
        cm[i,j]=temp

In [ ]:
pltu.plotConfusionMatrix(cm, labels, "confusion_matrix_2", "images_point_fourplus/")